Notebook created: 2018-03-29 04:30:33  
Generated from: _build_jl/jl/markov_asset.rst  

In [ ]:
using QuantEcon
using Plots
using LaTeXStrings

pyplot()

n = 25
mc = tauchen(n, 0.96, 0.25)
sim_length = 80

x_series = simulate(mc, sim_length; init=round(Int, n / 2))
g_series = exp.(x_series)
d_series = cumprod(g_series) # assumes d_0 = 1

series = [x_series g_series d_series log.(d_series)]
labels = [L"$X_t$" L"$g_t$" L"$d_t$" L"$log (d_t)$"]
plot(series, layout=4, labels=labels)

In [ ]:
n = 25  # size of state space
β = 0.9
mc = tauchen(n, 0.96, 0.02)

K = mc.p .* exp.(mc.state_values)'

I = eye(n)
v = (I - β * K) \  (β * K * ones(n, 1))

plot(mc.state_values,
    v,
    lw=2,
    ylabel="price-dividend ratio",
    xlabel="state",
    alpha=0.7,
    label=L"$v$")

In [ ]:
#=

@authors: Spencer Lyon, Tom Sargent, John Stachurski

=#

using QuantEcon

# A default Markov chain for the state process
ρ = 0.9
σ = 0.02
n = 25
default_mc = tauchen(n, ρ, σ)

mutable struct AssetPriceModel{TF<:AbstractFloat, TI<:Integer}
    β :: TF            # Discount factor
    γ :: TF            # Coefficient of risk aversion
    mc :: MarkovChain  # State process
    n :: TI            # Number of states
    g :: Function      # Function mapping states into growth rates
end

function AssetPriceModel(;β::AbstractFloat=0.96, γ::AbstractFloat=2.0,
                          mc::MarkovChain=default_mc, g::Function=exp)
    n = size(mc.p)[1]
    return AssetPriceModel(β, γ, mc, n, g)
end


"""
Stability test for a given matrix Q.
"""
function test_stability(ap::AssetPriceModel, Q::Matrix)
    sr = maximum(abs, eigvals(Q))
    if sr >= 1 / ap.β
        msg = "Spectral radius condition failed with radius = $sr"
        throw(ArgumentError(msg))
    end
end


"""
Computes the price-dividend ratio of the Lucas tree.

"""
function tree_price(ap::AssetPriceModel)
    # == Simplify names, set up matrices  == #
    β, γ, P, y = ap.β, ap.γ, ap.mc.p, ap.mc.state_values
    y = reshape(y, 1, ap.n)
    J = P .* ap.g.(y).^(1 - γ)

    # == Make sure that a unique solution exists == #
    test_stability(ap, J)

    # == Compute v == #
    I = eye(ap.n)
    Ones = ones(ap.n)
    v = (I - β * J) \ (β * J * Ones)

    return v
end

In [ ]:
using Plots, LaTeXStrings
pyplot()

γs = [1.2, 1.4, 1.6, 1.8, 2.0]
ap = AssetPriceModel()
states = ap.mc.state_values

lines = []
labels = []

for γ in γs
    ap.γ = γ
    v = tree_price(ap)
    label="γ = $γ"
    push!(labels, label)
    push!(lines, v)
end

plot(lines,
    labels=reshape(labels, 1, length(labels)),
    title="Price-dividend ratio as a function of the state",
    ylabel="price-dividend ratio",
    xlabel="state")

In [ ]:
"""
Computes price of a consol bond with payoff ζ

"""
function consol_price(ap::AssetPriceModel, ζ::AbstractFloat)
    # == Simplify names, set up matrices  == #
    β, γ, P, y = ap.β, ap.γ, ap.mc.p, ap.mc.state_values
    y = reshape(y, 1, ap.n)
    M = P .* ap.g.(y).^(-γ)

    # == Make sure that a unique solution exists == #
    test_stability(ap, M)

    # == Compute price == #
    I = eye(ap.n)
    Ones = ones(ap.n)
    p = (I - β * M) \ ( β * ζ * M * Ones)

    return p
end

In [ ]:
"""
Computes price of a perpetual call option on a consol bond.

"""
function call_option(ap::AssetPriceModel, ζ::AbstractFloat, p_s::AbstractFloat, ϵ=1e-7)

    # == Simplify names, set up matrices  == #
    β, γ, P, y = ap.β, ap.γ, ap.mc.p, ap.mc.state_values
    y = reshape(y, 1, ap.n)
    M = P .* ap.g.(y).^(-γ)

    # == Make sure that a unique console price exists == #
    test_stability(ap, M)

    # == Compute option price == #
    p = consol_price(ap, ζ)
    w = zeros(ap.n, 1)
    error = ϵ + 1
    while (error > ϵ)
        # == Maximize across columns == #
        w_new = max.(β * M * w, p - p_s)
        # == Find maximal difference of each component and update == #
        error = maximum(abs, w - w_new)
        w = w_new
    end

    return w
end

In [ ]:
ap = AssetPriceModel(β=0.9)
ζ = 1.0
strike_price = 40.0

x = ap.mc.state_values
p = consol_price(ap, ζ)
w = call_option(ap, ζ, strike_price)

plot(x, p, color="blue", lw=2, xlabel="state", label="consol price")
plot!(x, w, color="green", lw=2, label="value of call option")

In [ ]:
n = 5
P = 0.0125 .* ones(n, n)
P .+= diagm(0.95 .- 0.0125 .* ones(5))
s = [1.05, 1.025, 1.0, 0.975, 0.95]
γ = 2.0
β = 0.94
ζ = 1.0

In [ ]:
n = 5
P = 0.0125 .* ones(n, n)
P = P .+ diagm(0.95 - 0.0125 .* ones(5))
s = [0.95, 0.975, 1.0, 1.025, 1.05]  # state values
mc = MarkovChain(P, s)

γ = 2.0
β = 0.94
ζ = 1.0
p_s = 150.0

```none
150.0
```


In [ ]:
ap = AssetPriceModel(β=β, mc=mc, γ=γ, g=x -> x)

In [ ]:
v = tree_price(ap)
println("Lucas Tree Prices: $v\n")

```none
Lucas Tree Prices: [29.474,21.9357,17.5714,14.7252,12.7222]
```


In [ ]:
v_consol = consol_price(ap, 1.0)
println("Consol Bond Prices: $(v_consol)\n")

```none
Consol Bond Prices: [753.871,242.551,148.676,109.251,87.5686]
```


In [ ]:
w = call_option(ap, ζ, p_s)

In [ ]:
function finite_horizon_call_option(ap::AssetPriceModel,
                                    ζ::AbstractFloat,
                                    p_s::AbstractFloat,
                                    k::Int)

    # == Simplify names, set up matrices  == #
    β, γ, P, y = ap.β, ap.γ, ap.mc.p, ap.mc.state_values
    y = y'
    M = P .* ap.g.(y).^(- γ)

    # == Make sure that a unique console price exists == #
    test_stability(ap, M)

    # == Compute option price == #
    p = consol_price(ap, ζ)
    w = zeros(ap.n, 1)
    for i in 1:k
        # == Maximize across columns == #
        w = max.(β * M * w, p - p_s)
    end

    return w
end

In [ ]:
lines = []
labels = []
for k in [5, 25]
    w = finite_horizon_call_option(ap, ζ, p_s, k)
    push!(lines, w)
    push!(labels, "k = $k")
end
plot(lines, labels=reshape(labels, 1, length(labels)))